# Etivity 3 - Task 2: Regression
## Name: Brian Mortimer
## Student ID: 20258763

Open a new Jupyter notebook and name it etivity3_regression.ipynb. In this notebook, train three regression pipelines with Random Forest, Linear Regression and a third regressor of your choice as the final estimator, respectively, for predicting the value of `insurance_cost`.

Requirements:
- For each regressor, include data preparation and dimensionality reduction steps in the main pipeline.
- You can choose any regressor as the third one. Some options are SVR and MLPRegressor, but you are not limited to them.
- For the dimensionality reduction step use PCA, RFE and a third dimensionality reduction (incl. feature selection) technique in at least one pipeline.
- Use grid search for hyperparameter tuning and replicate the process in the example notebook Tutorial 3-2 - Regression and Dimensionality Reduction.ipynb to evaluate and compare the models you have trained and pick the best one.
- Summarise your experience in a markdown cell (max 150 words in a markdown cell).

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Functions
def load_insurance_data():
    """
    Load the insurance dataset from a CSV file.
    Returns a pandas DataFrame.
    """
    # Load the dataset
    df = pd.read_csv('insurance.csv')

    return df

In [3]:
# Load the dataset
df_original = load_insurance_data()
df = df_original.copy()

df.head()

,age,gender,bmi,children,smoker,region,insurance_cost
0,18,male,33.770,1,no,southeast,1725.55230
1,18,male,34.100,0,no,southeast,1137.01100
2,18,female,26.315,0,no,northeast,2198.18985
3,18,female,38.665,2,no,northeast,3393.35635
4,18,female,35.625,0,no,northeast,2211.13075


In [ ]:
# Pipeline
# Data preparation and dimensionality reduction


# Conclusion